# Otimizacao_Metaheuristica
<h2> Informações</h2>
Centro Federal de Educação Tecnológia Celso Suckow da Fonseca (CEFET/RJ)<br />
Trabalho da Disciplina: Otimização por Metaurísticas - Semestre 2022/02<br />

Artigo Base:
The Knapsack Problem with Forfeits
Cerulli, R., D’Ambrosio, C., Raiconi, A., Vitale, G. (2020). The Knapsack Problem with Forfeits. In: Baïou, M., Gendron, B., Günlük, O., Mahjoub, A.R. (eds) Combinatorial Optimization. ISCO 2020. Lecture Notes in Computer Science(), vol 12176. Springer, Cham. https://doi.org/10.1007/978-3-030-53262-8_22

<h2> Ações</h2><br/>

- [x] Viabilização de Instâncias
- [x] Algoritmos Construtivos (Guloso, Semi-Guloso, Aleatório)
- [X] Ajuste Algoritmo Construtivo com Penalidades
- [X] Framework GRASP
- [X] Algoritmo com Busca Local
- [X] Execução: 10x para cada Instância
- [X] Reportar Tabela: Melhor Solução
- [X] Reportar Tabela: Média da Solução
- [X] Reportar Tabela: Média do Tempo
- [X] Reportar Tabela: Coeficiente/Desvio de Variação das Soluções
- [X] Reportar Tabela: Coeficiente de Variação do Tempo

        

<h3>Classe para Leitura das Instâncias

In [1]:
# %history -g -f filename

In [2]:
import sys
import time

import numpy as np



class Instance:
    def __init__(self, filename):

        path = f"{filename}"

        f = open(path, "r")

        self.num_items, self.num_forfeits_pairs, self.budget = map(
            int, f.readline().split(" ")
        )

        f.close()

        # items definition
        items = []
        for i in range(self.num_items):
            items.append(i)

        self.items = np.array(items)

        line_counter = 1

        self.profits = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1
        
        self.weights = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1

        max = 2 * self.num_forfeits_pairs

        self.forfeit_cost_and_forfeits_pairs = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=max, usecols=(0, 1)
        ).tolist()

        self.forfeits_costs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 == 0
        ]

        self.forfeits_pairs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 != 0
        ]

        # remove the first element of the sublist
        for i in range(len(self.forfeits_costs)):
            self.forfeits_costs[i].pop(0)

        # transform list of lists into a single list - flatten
        self.forfeits_costs = [
            item for sublist in self.forfeits_costs for item in sublist
        ]

        self.forfeits_costs = np.array(self.forfeits_costs)
        self.forfeits_pairs = np.array(self.forfeits_pairs)      

<h2>Montando a Matriz de Penalidades

In [3]:
def calculate_penalty(items,forfeits_pairs, forfeits_costs):
    mD=np.zeros((int(len(items)),int(len(items))))       
    
    for index, pair in enumerate(forfeits_pairs):
        mD[int(pair[0]),int(pair[1])]=forfeits_costs[index]                
    return mD

<h1>Algoritmo Construtivo (Guloso, Aleatório e Semi-Guloso)</h1>

In [4]:
import random as rd

def greedyalgorithm(items, weights, profits, budget, forfeits, forfeits_costs, alpha,mD):    
    
    cap=budget    

    #print(sorted_items)    
    solution=[]
    scost=[]
    sweights=[]
    cost=0
    index=0
    
    if alpha == 0:  # totalmente aleatório 
        remaining_items=zip(items,weights,profits)
        sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
        #print(tuple(items))        
        
        rd_index=rd.choice(range(0,len(sorted_items))) 
        candidate=sorted_items[rd_index][0]
        
        while cap-weights[candidate]>0:                                        
                cap=cap-weights[candidate]
                solution.append(candidate) 
                
                
                penalidade=sum(mD[candidate][solution])                
                #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                    
                cost=cost+profits[candidate]-penalidade
                
                scost.append(profits[candidate]-penalidade)
                
                sweights.append(weights[candidate])
                
                sorted_items.remove(sorted_items[rd_index])
                rd_index=rd.choice(range(0,len(sorted_items)))  
                candidate=sorted_items[rd_index][0]
        
                    
    else:
            if alpha == 1: #totalmente guloso
                remaining_items=zip(items,weights,profits)
                #print(tuple(items))
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                #sorted_items=sorted(remaining_items, key= lambda x:x[2]/x[1], reverse=True)
                #print(sorted_items)
                candidate=sorted_items[0][0]
                while cap-weights[candidate]>0:                                                  
                    
                    #calcula a nova penalidade para o item a ser inserido no conjunto e reordena                                      
                    #sorted_items=sorted(sorted_items, key= lambda x:(x[2]/x[1])-sum(mD[x[0]][solution]), reverse=True)                                        
                    
                    cap=cap-weights[candidate]
                    solution.append(candidate)  
                    
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[0][0]} é {penalidade}")
                    
                    #cost=cost+sorted_items[0][2]
                    cost=cost+profits[candidate]-penalidade
                    #cost=cost+sorted_items[index][2]-sum(mD[sorted_items[index][0]][solution])
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                    
                    #sorted_items=sorted(sorted_items, key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        
                    sorted_items.remove(sorted_items[0])
                    
                    if len(sorted_items)>=0:
                        #sorted_items=sorted(sorted_items, key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        
                        candidate=sorted_items[0][0]
                    #sorted_items.remove(sorted_items[index])
                    #index=index+1  
                    
            else: # semi-guloso
                remaining_items=zip(items,weights,profits,profits/weights)
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                
                
                # o cálculo dos limites da lcr
                hmax=sorted_items[0][3]
                hmin=sorted_items[-1][3]

                    #ub = hmax + alpha * (hmin - hmax)
                lb = hmin                            
                ub=hmax + alpha * (hmin - hmax)                  

                    #lista restrita de candidatos
                    #lcr=list(filter(lambda x: x[3]<=ub and x[3] >= lb, sorted_items))                                
                lcr=list(filter(lambda x: x[3]>=ub, sorted_items))
                rd_index=rd.choice(range(0,len(lcr))) 
                candidate=lcr[rd_index][0]
                
                #print(candidate)
                while cap-weights[candidate]>0:  
                                                                          
                    cap=cap-weights[candidate]
                    solution.append(candidate)    
                    #print(f" Solution: {solution}")
                                        
                    
                    #penalidade
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                                        
                    cost=cost+profits[candidate]-penalidade                                        
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                
                    #removendo o item já inserido
                    sorted_items.remove(sorted_items[sorted_items.index(lcr[rd_index])])                    
               
                                          
                    #sorted_items.sort(key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        

                     # o cálculo dos limites da lcr
                    hmax=sorted_items[0][3]
                    hmin=sorted_items[-1][3]
                    
                    #ub = hmax + alpha * (hmin - hmax)
                    lb = hmin                            
                    ub=hmax + alpha * (hmin - hmax)
                 
                    #lista restrita de candidatos                                
                    lcr=list(filter(lambda x: x[3]>=ub, sorted_items))

                    #print(f"LCR: {lcr}")
                    rd_index=rd.choice(range(0,len(lcr))) 
                    candidate=lcr[rd_index][0]
                                
    
    return solution,cost,cap,scost,sweights,sorted_items

### Teste Construtivo (Guloso, Aleatório, Semi-guloso)

In [5]:
pasta = 'instances/'
filename = pasta+"500/kpf_1.txt"
print(filename)
problem_instance = Instance(filename)

mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
# print(sum(mD[321][:]))
# print(sum(mD[321][11]))
start_time = time.time()
alpha=0.7
solution,cost,cap,scost,sweights,sorted_items = greedyalgorithm(
    problem_instance.items,
    problem_instance.weights,
    problem_instance.profits,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs,
    alpha,
    mD,
)

end_time = time.time()

wall_time = end_time - start_time


# print(wall_time)
# print(f"Custo {cost}")
# print(f"Solução {solution}")



instances/500/kpf_1.txt


<h1>Algoritmo de Busca Local

Primeira Melhora
Este método de melhoria explora a vizinhança de troca,ou seja, ele troca um elemento aleatório na mochila com um elemento de fora, respeitando o limite de capacidade, e buscando uma melhoria na função que objetiva o maior lucro.

## Teste Busca Local + Construtivo

In [55]:
import random
def local_search(solution, items, weights, profits, solution_cost, budget, forfeits_costs, forfeits_pairs, mD):
    # vizinhança tres elementos consecutivos

    solution_set = set(solution)
    items_set = set(items)

    previous_cost = solution_cost

    # convert forfeits pair in a forfeits pair set
    forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

    # items not in solution yet
    remaining_items = items_set.difference(solution_set)
    
    improved = False
    ls_cost = int()
    while improved == False:
        

        # choosing a random element to enter in solution
        # enter_item = random.choice(tuple(remaining_items)) -> antes o item era escolhida aleatoriamente na lista de restantes
        for item in remaining_items: # será iterado apenas uma vez por todos itens que não estão na solução
            enter_item = item

            # choosing a random element to leave the solution
            leave_item = random.choice(tuple(solution_set))

            # exchange elements
            if profits[enter_item] >= profits[leave_item]:
                solution_set.remove(leave_item)
                solution_set.add(enter_item)

                # calculate solution weights to validate those items that is less than the budget
                solution_weights = 0
                for item in solution_set:
                    solution_weights += weights[item]
                # print(f"weights: {solution_weights}") #debug

                if solution_weights <= budget: #feasibility
                
                    # 1. calculate items profits (update all items profits - leave and enter items)
                    ls_cost = 0
                    for item in solution_set:
                        ls_cost += profits[item]

                    # 2. subtract enter items penalties 
                    for i in solution_set:
                        if (i, enter_item) in forfeits_pairs_set:
                            ls_cost -= mD[i][enter_item]

                        if (enter_item, i) in forfeits_pairs_set:
                            ls_cost -= mD[enter_item][i]

                    # print(f"cost_ls: {ls_cost}") #debug

                if ls_cost > solution_cost:
                    print('improved')
                    improved = True


    return ls_cost, list(solution_set)

In [56]:
pasta = 'instances/'
filename = pasta+"500/kpf_1.txt"
print(filename)
problem_instance = Instance(filename)

mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
start_time = time.time()
alpha=1 #guloso
solution_greedy,cost_greedy,cap,scost,sweights,sorted_items = greedyalgorithm(
    problem_instance.items,
    problem_instance.weights,
    problem_instance.profits,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs,
    alpha,
    mD,
)

print(f"Custo construtivo: {cost_greedy}")
print(f"Solução construtivo: {solution_greedy}")

cost_ls, solution_ls = local_search(
    solution_greedy, 
    problem_instance.items, 
    problem_instance.weights, 
    problem_instance.profits, 
    cost_greedy,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs, 
    mD)

end_time = time.time()

wall_time = end_time - start_time


print(wall_time)
print(f"Custo Busca Local: {cost_ls}")
print(f"Solução Busca Local: {solution_ls}")

instances/500/kpf_1.txt
Custo construtivo: 1767.0
Solução construtivo: [271, 192, 72, 159, 196, 471, 77, 174, 456, 90, 116, 478, 130, 40, 63, 187, 66, 310, 346, 95, 353, 406, 463, 22, 99, 241, 296, 446, 464, 5, 173, 204, 450, 14, 219, 257, 295, 186, 481, 17, 79, 117, 211, 300, 402, 145, 135, 170, 317, 482, 491, 134, 473, 127, 290, 21, 39, 87, 171, 229, 253, 488, 480, 161, 227, 239, 256, 309, 362, 367, 457, 476, 44, 101, 427, 495, 38, 226, 337, 382, 311, 314, 18, 224, 37, 163, 254, 73, 356, 202, 234, 380, 327, 19, 265, 54, 150, 262, 364, 395, 359, 425, 9, 59, 82, 178, 220, 92, 247, 368, 409, 315, 89, 370, 151, 423, 140, 329, 412, 35, 93, 96, 111, 142, 162, 177, 237, 266, 297, 344, 366, 373, 378, 389, 403, 413, 440, 454, 240, 312, 8, 455, 42, 328, 434, 91, 109, 225, 363, 47, 11, 209, 55, 246, 264, 339, 386, 60, 188, 245, 181, 199, 484, 4, 396, 437, 128, 182, 269, 164, 102, 213, 288, 348, 411, 486, 16, 29, 477, 417, 452, 154, 157, 138, 41, 146, 203, 201, 52, 56, 94, 175, 248, 305, 306, 41

<h1>GRASP

In [ ]:
def grasp(max_iterations,items, weights, profits, budget, forfeits_pairs, forfeits_costs, alpha,mD):
    
    solution_final=0
    cost_final=0;
    cost_aux=0;
    while max_iterations>=0:
        solution,cost,cap,scost,sweights,sorted_items = greedyalgorithm(
        items,
        weights,
        profits,
        budget,
        forfeits_pairs,
        forfeits_costs,
        alpha,
        mD,)
        
        s1,cost1,scost1,scap1,sweights1,sorted_items1= busca_local(solution,cost,scost,sweights,sorted_items,cap)
        
        if cost>cost1:
            cost_aux=cost
            solution_aux=solution
        else:
            cost_aux=cost1
            solution_aux=s1
        print(solution_aux)
        print(cost_aux)
        
        if cost_aux>cost_final:
            cost_final=cost_aux
            solution_final=solution_aux
        
        max_iterations=max_iterations-1
    

    return solution_final,cost_final

# ILS - Iterated Local Search

In [57]:
import random

def perturbation(solution:list(), items) -> list():
    # transform solution list to a set
    solution_set = set(solution)
    items = set(items)

    # items not in solution yet
    remaining_items = items.difference(solution_set)

    for i in range(1,6): # change 5 random numbers
        # choosing a random element to enter in solution
        enter_item = random.choice(tuple(remaining_items))
        # print(f'enter:{enter_item}')
        
        # choosing a random element to leave the solution
        leave_item = random.choice(tuple(solution_set))
        # print(f'leave:{leave_item}')

        # exchange elements
        solution_set.remove(leave_item)
        solution_set.add(enter_item)

    return list(solution_set)
    


def ILS(items, weights, profits, budget, forfeits_costs, forfeits_pairs, alpha, mD):
    # generate initial solution
    s0,cost0,cap,scost,sweights,sorted_items = greedyalgorithm(
        items, weights, profits, budget, forfeits_costs, forfeits_pairs, alpha, mD)
    
    print(f"cost construtivo: {cost0}")

    #local search
    cost_star, s_star = local_search(s0, items, weights, profits, cost0, budget, forfeits_costs, forfeits_pairs, mD)

    print(f"cost busca local: {cost_star}")
    improved = False
    while improved == False:
        # perturbation in solution
        s_line = perturbation(s_star, items)

        cost_star_line, s_star_line = local_search(s_star, items, weights, profits, cost_star, budget, forfeits_costs, forfeits_pairs, mD)

        print(f"cost star_line: {cost_star_line}")

        # acceptance criterion
        if cost_star_line > cost_star:
            improved = True

    return s_star_line, cost_star_line

## Teste ILS

In [59]:
pasta = 'instances/'
filename = pasta+"500/kpf_1.txt"
print(filename)
problem_instance = Instance(filename)

mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
start_time = time.time()
alpha=1 #guloso

start_time = time.time()

solution, cost = ILS(problem_instance.items, 
    problem_instance.weights,
    problem_instance.profits,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs,
    alpha,
    mD,)

end_time = time.time()

wall_time = end_time - start_time


print(wall_time)
print(f"Custo ILS: {cost}")
print(f"Solução {solution}")


instances/500/kpf_1.txt
cost construtivo: 1767.0
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
cost busca local: 2312.0
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
improved
cost star_line: 2384.0
0.07778811454772949
Custo ILS: 2384.0
Solução [5, 21, 37, 43, 52, 54, 66, 74, 81, 86, 87, 88, 93, 97, 98, 105, 108, 113, 116, 117, 120, 126, 135, 140, 145, 148, 151, 157, 160, 163, 171, 173, 178, 180, 182, 186, 188, 196, 197, 204, 208, 215, 221, 227, 229, 232, 234, 239, 240, 244, 245, 254, 257, 261, 266, 267

<h2>Funções Auxiliares (Caso Necessário)

<h3>Função para Cálculo do Lucro dos itens constantes como solução(incluídos na mochila)

In [ ]:
def calcCost(solution, profits):
    cost = 0
    for sol in solution:
        sol = int(sol)
        cost = cost + profits[sol]
    return cost

<h3> Função Para Cálculo da Capacidade da Mochila

In [ ]:
def calcb(solution, w):
    bres = 0
    for sol in solution:
        sol=int(sol)
        bres = bres + int(w[sol])
    return bres

<h2> Iniciando a Execução

<h3>Buscando os Arquivos das Instâncias

In [ ]:
import os
pasta = 'instances/500/'
instancias=[]
name_instancias=[]
for diretorio, subpastas, arquivos in os.walk(pasta):
    for arquivo in arquivos:
        instancias.append(os.path.join("500/", arquivo))
        name_instancias.append(arquivo.split(".")[0])
print(instancias)
print(name_instancias)

<h2> Lista de Algoritmos para Execução

In [ ]:
import numpy as np
algoritmos = np.array(["Guloso","Aleatorio","SemiGuloso","GRASP"])
#algoritmos = np.array(["Guloso","Aleatorio","SemiGuloso"])

<h3>Criando o Diretório para guardar os resultados

In [ ]:
# pasta_result = '/kaggle/working/result'
# if not os.path.isdir(pasta_result): # vemos de este diretorio ja existe
#     os.mkdir(pasta_result) # aqui criamos a pasta caso nao exista

In [ ]:
#Apagando o Diretório dos Resultados

#import shutil
#shutil.rmtree("/kaggle/working/result")
#instancias[3]

<h1>Realizando para todas as instâncias e algoritmos da Lista

In [ ]:
mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
pasta = '../input/instances/instances/'
pasta_result = "resultados-notebook"
#filename = pasta+"500/kpf_1.txt"
#problem_instance = Instance(filename)
#pasta = 'instances/'
for inst in instancias:    
    #filename = pasta+inst
    print(filename)
    problem_instance = Instance(filename)
    mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
    for alg in algoritmos:
        if alg=="Aleatorio":
                alpha=float(0)
        else:
            if alg=="Guloso":
                alpha=float(1)
            else:
                if alg=="SemiGuloso" or alg=="GRASP":
                    alpha = float(0.7)                
                        
        for i in range(1, 11):
        #for i in range(1, 2):
            if alg!="GRASP":            
                start_time = time.time()                 
                solution,cost,cap,scost,sweights,sorted_items = greedyalgorithm(
                #solution,cost,scost,cap,sweights1,sorted_items= greedyalgorithm(
                    problem_instance.items, problem_instance.weights, problem_instance.profits,
                    problem_instance.budget, problem_instance.forfeits_pairs, problem_instance.forfeits_costs,
                    alpha,
                    mD,)
                end_time = time.time()
                wall_time = end_time - start_time
            else:
                max_iterations=50
                start_time = time.time()                                 
                solution,cost=grasp(
                    max_iterations,
                    problem_instance.items,
                    problem_instance.weights,
                    problem_instance.profits,
                    problem_instance.budget,
                    problem_instance.forfeits_pairs,
                    problem_instance.forfeits_costs,
                    alpha,
                    mD,)
                end_time = time.time()
                wall_time = end_time - start_time
                

            #cost=calcCost(solution,problem_instance.profits)
            f = open(f"{pasta_result}/resultados_algoritmo_{alg}_instancia_{name_instancias[instancias.index(inst)]}.txt", "a")

            f.write(f"execucao_{i} para {inst}:\n")
            print(f"execucao_{i} para {inst}:\n")
            f.write(f"-> alpha: {alpha}\n")
            f.write(f"-> custo solucao : {cost}\n")
            f.write(f"-> tempo: {wall_time}\n")
            #f.write(f"-> solucao: {solution}\n")

            f.write(f"\n")
            f.write(f"\n")
            f.close()
            
            if alg!="GRASP":
            #if alg=="GRASP":
                start_time1 = time.time()                 
                s1,cost1,scost1,scap1,sweights1,sorted_items1= busca_local(solution,cost,scost,sweights,sorted_items,cap)
                end_time1 = time.time()
                wall_time1 = end_time1 - start_time1

                fbl = open(f"{pasta_result}/resultados_busca_local_algoritmo_{alg}_instancia_{name_instancias[instancias.index(inst)]}.txt", "a")
                fbl.write(f"Busca Local execucao_{i} para {inst}:\n")
                print(f"BUSCA LOCAL execucao_{i} para {inst}:\n")
                fbl.write(f"-> alpha: {alpha}\n")
                fbl.write(f"-> custo solucao : {cost1}\n")
                fbl.write(f"-> tempo: {wall_time1+wall_time}\n")
                #fbl.write(f"-> solucao: {s1}\n")
                fbl.write(f"\n")
                fbl.close()

<h1>Extraindo os resultados Obtidos

In [ ]:
pasta = '../input/instances/instances/'
from tabulate import tabulate
d=[]
d2=[]
for inst in instancias:   
    filename = pasta+inst
    problem_instance = Instance(filename)
    for alg in algoritmos:
        if alg=="Guloso":
            nalg=1
        elif alg=="Aleatorio":
            nalg=0
        elif alg=="SemiGuloso":
            nalg=2
        elif alg=="GRASP":
            nalg=3
        

        f = open(f"{pasta_result}/resultados_algoritmo_{alg}_instancia_{name_instancias[instancias.index(inst)]}.txt", "r")
        arquivo = f.readlines()                
        
        if alg!="GRASP":
        #if alg=="GRASP":
            fbl = open(f"{pasta_result}/resultados_busca_local_algoritmo_{alg}_instancia_{name_instancias[instancias.index(inst)]}.txt", "r")        
            arquivobl = fbl.readlines()
            custo_bl=[]
            tempo_bl=[]
            local_custo_bl=0
            local_tempo_bl=0
        
        local_custo=0        
        local_tempo=0        
   
        custo=[]
        tempo=[]
        
        for i in range(0, 10):
        #for i in range(0, 1):
            
            local_custo=(arquivo[(i*6)+2].split("-> custo solucao : ")[1]).strip('\n')
            #print(arquivo[(i*6)+3].split("-> tempo: ")[1])
            local_tempo=(arquivo[(i*6)+3].split("-> tempo: ")[1]).strip('\n')
            
            d.append((alg,name_instancias[instancias.index(inst)], i+1,local_custo,local_tempo))
            #custo=custo+float(local_custo)
            #tempo=tempo+float(local_tempo)
            custo.append(float(local_custo))
            tempo.append(float(local_tempo))
            
            
            if alg!="GRASP":
            #if alg=="GRASP":
            
                local_custo_bl=(arquivobl[(i*5)+2].split("-> custo solucao : ")[1]).strip('\n')
                #print(arquivobl[(i*5)+3].split("-> tempo: ")[1])
                local_tempo_bl=(arquivobl[(i*5)+3].split("-> tempo: ")[1]).strip('\n')

                d.append((alg+"_BL",name_instancias[instancias.index(inst)], i+1,local_custo_bl,local_tempo_bl))
                
                custo_bl.append(float(local_custo_bl))
                tempo_bl.append(float(local_tempo_bl))
            
            
        
        #media
        #print(custo)
        media_custo=np.mean(custo)
        media_tempo=np.mean(tempo)
        #variancia
        coefvar_custo=np.var(custo)
        coefvar_tempo=np.var(tempo)
        
        
        #desvio padrao
        stdvar_custo=np.std(custo)
        stdvar_tempo=np.std(tempo)
        
        if alg!="GRASP":
        #if alg=="GRASP":
            media_custo_bl=np.mean(custo_bl)
            media_tempo_bl=np.mean(tempo_bl)
            coefvar_custo_bl=np.var(custo_bl)
            coefvar_tempo_bl=np.var(tempo_bl)
            stdvar_custo_bl=np.std(custo_bl)
            stdvar_tempo_bl=np.std(tempo_bl)
            d2.append((alg+"_BL",name_instancias[instancias.index(inst)],media_custo_bl,coefvar_custo_bl/100,stdvar_custo_bl,
                  media_tempo_bl,coefvar_tempo_bl,stdvar_tempo_bl))

        
        
        d2.append((alg,name_instancias[instancias.index(inst)],media_custo,coefvar_custo/100,stdvar_custo,
                   media_tempo,coefvar_tempo,stdvar_tempo))
                  

print(tabulate(d, headers=["Algoritmo","Instancia","Execucao","Custo","Tempo"]))

print("\n\n\n")

c = open(f"{pasta_result}/resultado_consolidado.txt", "a")
resultc=tabulate(d, headers=["Algoritmo","Instancia","Execucao","Custo","Tempo"])
c.write("\n\n")
c.write(resultc)
print(resultc)
c.close()

<h1>Apresentando os Resultados Obtidos, Ordenando pelo Melhor Custo

In [ ]:
import pandas as pd
df = pd.DataFrame(d2,columns=["Algoritmo","Instancia","Md Custo","Coef VAR(%)-Custo ","Desv Pd-Custo","Md Tempo","Coef VAR(%)-TEMPO","Desv Pd-TEMPO"])

In [ ]:
 def color_row3(x):
    bc = []
    d=""
    c='background-color: '
    for i in range(0,len(x)):
        if (i+7)%7==0:
            if c=='background-color:  #c2c2d6':
                c='background-color:  #b3ffff'
            else:
                c='background-color:  #c2c2d6'
            d=d+';color: red; font-weight:bold'
            #d=d+';color: black; font-weight:bold'
        else:
            d=d+';color:black'        
        bc.append(c+d)        
        d=""
    return bc
#color_row3(d2)
df.sort_values(['Instancia','Md Custo'],ascending=False).style.apply(color_row3)

Resultado Totalizado

In [ ]:
d3=[]

for ninst in name_instancias:        
    df_local=df[df["Instancia"]==ninst]
       
    #d3.append((ninst,media_custo,coefvar_custo/100,stdvar_custo,media_tempo,coefvar_tempo,stdvar_tempo))
    
    media_custo=np.mean(df_local['Md Custo'])
    media_tempo=np.mean(df_local['Md Tempo'])

    #variancia
    coefvar_custo=np.var(df_local['Md Custo'])
    coefvar_tempo=np.var(df_local['Md Tempo'])

    #desvio padrao
    stdvar_custo=np.std(df_local['Md Custo'])
    stdvar_tempo=np.std(df_local['Md Tempo'])
    d3.append((ninst,media_custo,coefvar_custo/100,stdvar_custo,
                   media_tempo,coefvar_tempo/100,stdvar_tempo))
        

    
df2 = pd.DataFrame(d3,columns=["Instancia","Md Custo","Coef VAR(%)-Custo ","Desv Pd-Custo","Md Tempo","Coef VAR(%)-TEMPO","Desv Pd-TEMPO"])

df2.sort_values(['Md Custo'],ascending=False)
#print(custo)
#print(tempo)

<h1>Representações Gráficas

In [ ]:
import matplotlib.pyplot as plt
#Coef VAR(%)-Custo	Desv Pd-Custo	Md Tempo	Coef VAR(%)-TEMPO	Desv Pd-TEMPO
df_kpf_1=df[df['Instancia']=='kpf_1']

ax = plt.gca()
#df_kpf_1.plot(kind='line', x='Algoritmo', y='Md Custo', color='blue', ax=ax, figsize=(10,6))
df_kpf_1.plot(kind='line', x='Algoritmo', y='Desv Pd-Custo', color='green', ax=ax, figsize=(10,6))
#df_kpf_1.plot(kind='line', x='Algoritmo', y='Coef VAR(%)-TEMPO', color='yellow', ax=ax, figsize=(10,6))


plt.show()



In [ ]:
ax = plt.gca()
#df_kpf_1.plot(kind='line', x='Algoritmo', y='Md Tempo', color='blue', ax=ax, figsize=(10,6))
#df_kpf_1.plot(kind='line', x='Algoritmo', y='Desv Pd-Custo', color='green', ax=ax, figsize=(10,6))
df_kpf_1.plot(kind='line', x='Algoritmo', y='Coef VAR(%)-TEMPO', color='blue', ax=ax, figsize=(10,6))
plt.show()



In [ ]:

fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(df[df['Algoritmo']=='SemiGuloso_BL']['Instancia'], df[df['Algoritmo']=='SemiGuloso_BL']['Md Custo'], label='SemiGuloso_BL')
ax.plot(df[df['Algoritmo']=='SemiGuloso']['Instancia'], df[df['Algoritmo']=='SemiGuloso']['Md Custo'], label='SemiGuloso')
ax.plot(df[df['Algoritmo']=='Guloso_BL']['Instancia'], df[df['Algoritmo']=='Guloso_BL']['Md Custo'], label='Guloso_BL')
ax.plot(df[df['Algoritmo']=='Guloso']['Instancia'], df[df['Algoritmo']=='Guloso']['Md Custo'], label='Guloso')
ax.plot(df[df['Algoritmo']=='Aleatorio']['Instancia'], df[df['Algoritmo']=='Aleatorio']['Md Custo'], label='Aleatorio')
ax.plot(df[df['Algoritmo']=='Aleatorio_BL']['Instancia'], df[df['Algoritmo']=='Aleatorio_BL']['Md Custo'], label='Aleatorio_BL')
ax.plot(df[df['Algoritmo']=='GRASP']['Instancia'], df[df['Algoritmo']=='GRASP']['Md Custo'], label='GRASP')
ax.set_xlabel('Instancias')
ax.set_ylabel('Lucro')
ax.set_title("Lucro por Algoritmo e Instância")
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
#ax.plot(df[df['Algoritmo']=='SemiGuloso_BL']['Instancia'], df[df['Algoritmo']=='SemiGuloso_BL']['Md Tempo'], label='SemiGuloso_BL')
ax.plot(df[df['Algoritmo']=='SemiGuloso']['Instancia'], df[df['Algoritmo']=='SemiGuloso']['Md Tempo'], label='SemiGuloso')
#ax.plot(df[df['Algoritmo']=='Guloso_BL']['Instancia'], df[df['Algoritmo']=='Guloso_BL']['Md Tempo'], label='Guloso_BL')
ax.plot(df[df['Algoritmo']=='Guloso']['Instancia'], df[df['Algoritmo']=='Guloso']['Md Tempo'], label='Guloso')
ax.plot(df[df['Algoritmo']=='Aleatorio']['Instancia'], df[df['Algoritmo']=='Aleatorio']['Md Tempo'], label='Aleatorio')
ax.plot(df[df['Algoritmo']=='GRASP']['Instancia'], df[df['Algoritmo']=='GRASP']['Md Tempo'], label='GRASP')
#ax.plot(df[df['Algoritmo']=='Aleatorio_BL']['Instancia'], df[df['Algoritmo']=='Aleatorio_BL']['Md Tempo'], label='Aleatorio_BL')
ax.set_xlabel('Instancias')
ax.set_ylabel('Tempo')
ax.set_title("Tempo por Algoritmo e Instância")
ax.legend()
plt.show()